In [93]:
import pandas as pd
import requests
from datetime import datetime
import time
from bs4 import BeautifulSoup

In [2]:
def get_blocknumber_by_timestamp(date_string):
    
    # Function retrieving block number by date in format YYY    
    
    # convert date string to timestamp
    date_object = datetime.strptime(date_string, '%Y-%m-%d %H:%M')
    timestamp = int(time.mktime(date_object.timetuple()))
    
    # Find closes block to give timestamp (date_string)
    # Set the API endpoint
    url = 'https://api.etherscan.io/api'

    # Set the API parameters
    params = {
        'module': 'block',
        'action': 'getblocknobytime',
        'timestamp': timestamp,
        'closest': 'before', # the closest available block to the provided timestamp, either BEFORE or AFTER - https://docs.etherscan.io/api-endpoints/blocks#get-block-number-by-timestamp
        'apikey': 'H7X31IMU7EMEEBDE4633U8F73SNMYDH5AZ'
    }

    # Send a GET request to the API endpoint with the defined parameters
    response = requests.get(url, params=params)

    # Parse the response JSON data
    data = response.json()['result']

    return data

get_blocknumber_by_timestamp("2023-03-10 12:00")

'16797293'

In [34]:
# Creating dictionary from etherscan labels
eth_labels = pd.read_csv('eth_addresses.csv')

address_dict = {}
for index, row in eth_labels.iterrows():
    #address_dict[row['Address']] = (str(row['Account Type'])+' -'+str(row['Name']))
    address_dict[row['Address']] = (row['Account Type'])

In [165]:
def search_for_address_label(address):

    url = f'https://etherscan.io/address/{address}'

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')

    label_tag = soup.find_all('a', class_='badge bg-white hover:bg-secondary border border-dark dark:border-white border-opacity-15 text-dark text-nowrap fw-medium transition-all rounded-pill py-1.5 px-2')

    public_tag = soup.find_all('span', title='Public Name Tag (viewable by anyone)')
    
    if label_tag != []:
        label_tag = label_tag[0].text.strip()
    
    if public_tag != []:
        public_tag = public_tag[0].text.strip()
    
    return(label_tag)
        
search_for_address_label('0xa88800cd213da5ae406ce248380802bd53b47647')

'1inch'

In [192]:
def get_token_transactions_by_dates(token_address, start_date, end_date):

    # Set the API endpoint
    url = 'https://api.etherscan.io/api'

    token_address = '0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF'

    # Set the API parameters
    params = {
        'module': 'account',
        'action': 'tokentx',
        'contractaddress': token_address,
        'startblock': start_date,
        'endblock': end_date,
        'sort':'desc',
        'apikey': 'H7X31IMU7EMEEBDE4633U8F73SNMYDH5AZ'
    }

    # Send a GET request to the API endpoint with the defined parameters
    response = requests.get(url, params=params)

    # Parse the response JSON data
    data = response.json()

    # Extract the transaction data from the response
    transactions = data['result']
    
    unique_token_receivers = list(set([address['to'] for address in transactions]))

    return [transactions,unique_token_receivers]

In [193]:
# Process for unique receivers of a token for a given date

token_address = '0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF'
start_date = "2023-03-21 07:00"
end_date = "2023-03-21 10:30"

token_info = get_token_transactions_by_dates(token_address, start_date, end_date)
token_transacations = token_info[0]
token_unique_receivers = token_info[1]

In [208]:
input_data = pd.read_csv("input.csv")

for index, row in input_data.iterrows():
    get_token_transactions_by_dates(row[0], row[1], row[2])[1]

0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF  2023-03-21 07:00  2023-03-21 10:30
0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF  2023-03-20 07:00  2023-03-20 10:30
0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF  2023-03-11 07:00  2023-03-11 10:30
0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF  2023-03-21 07:00  2023-03-21 10:30
0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF  2023-03-21 07:00  2023-03-21 10:30
0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF  2023-03-21 07:00  2023-03-21 10:30


In [185]:
# Collect labels from addresses
addies_labels_list = {}

for addie in unique_token_receivers[:20]:
    addies_labels_list[addie] = search_for_address_label(addie)

In [186]:
unique_token_receivers_replaced = list(unique_token_receivers)

for i in range(len(unique_token_receivers_replaced)):
    if unique_token_receivers_replaced[i] in addies_labels_list:
        unique_token_receivers_replaced[i] = addies_labels_list[unique_token_receivers_replaced[i]]